## Solving puzzles with Python

The puzzles in [this pdf](/files/uspc-09.pdf) are from US Puzzle Contest 2009. PDF password is `barMd345`.

In [45]:
from itertools import product, combinations, permutations, tee, islice, repeat, combinations_with_replacement
from collections import Counter
from random import choice
from constraint import Problem, BacktrackingSolver

#### 1. Battleships (Moshe Rubin) - 5 points

> Locate the position of the 10-ship fleet in
 the grid. The fleet is shown below: one 
4-unit battleship, two 3-unit cruisers, 
three 2-unit destroyers, and four 1-unit 
submarines. Each segment of a ship occupies a single cell. Ships are oriented 
either horizontally or vertically, 
and they do not touch each other, not even 
diagonally. The numbers on the right and bottom edges of the grid reveal the total 
number of ship segments that appear in each respective row or column. (For 
solving purposes, ignore the letters above and to the left of the grid.)

<img src='/files/2009_1_battleships_fleet.png' />
<img src='/files/2009_1_battleships_grid.png' />

> Answer: Enter the coordinates of the three missing 1-unit
 submarines (using the letters above and to the left of 
the grid). 

In [2]:
# ship pseudo-graphic:
# h - ltr head
# t - ltr head
# H - up-down head
# T - up-down tail
# b - ltr body segment
# B - up-down body segment
# s - 1-unit submarine
# W - water (empty cell)'

In [82]:
from string import ascii_uppercase
from operator import eq
import sys

grid_size = 10
possible_values = [f'{i:02}' for i in range(grid_size*grid_size)]
visibility = {'T': 1, 'H': 1, 'B': 1, 'S': 1, 'W': 0}
row_visible_parts = [4, 0, 2, 1, 1, 2, 0, 5, 0, 5]
col_visible_parts = [2, 2, 1, 1, 1, 2, 2, 1, 6, 2]
known_parts = {'00': 's', '20': 'W', '92': 'B', '97': 'H'}
fleet_size = 10

b2 = list(combinations_with_replacement(['hbbt', 'HBBT'], 1))
b1 = list(combinations_with_replacement(['hbt', 'HBT'], 2))
b0 = list(combinations_with_replacement(['ht', 'HT'], 3))
subs = ['s', 's', 's', 's']

fleet_permutations = [list(chain(x, y, z, subs)) for x in b2 for y in b1 for z in b0]

for fleet in fleet_permutations:
    p = Problem()
    p.addVariables(range(fleet_size), possible_values)
    p.addConstraint(AllDifferentConstraint(), range(fleet_size))
    print(p.getSolution())

{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}
{0: '99', 1: '98', 2: '97', 3: '96', 4: '95', 5: '94', 6: '93', 7: '92', 8: '91', 9: '90'}

In [35]:
from string import ascii_uppercase
from operator import eq
import sys

size = 10
possible_values = 'THBSW'
visibility = {'T': 1, 'H': 1, 'B': 1, 'S': 1, 'W': 0}
grid_variables = [f'{i:02}' for i in range(size*size)]

p = Problem()
p.addVariables(grid_variables, possible_values)

def visible_parts(*cells, val=None):
    return sum(map(visibility.get, cells)) == val

row_visible_parts = [4, 0, 2, 1, 1, 2, 0, 5, 0, 5]
for i, val in enumerate(row_visible_parts):
    col = [f'{i}{j}' for j in range(size)]
    p.addConstraint(partial(visible_parts, val=val), col)
    if val == 0:
        p.addConstraint(lambda *cell: all(map(partial(eq, 'W'), cell)), col)

col_visible_parts = [2, 2, 1, 1, 1, 2, 2, 1, 6, 2]
for j, val in enumerate(col_visible_parts):
    row = [f'{i}{j}' for i in range(size)]
    p.addConstraint(partial(visible_parts, val=val), row)

known_parts = {'00': 'S', '20': 'W', '92': 'B', '97': 'T'}
for cell, part in known_parts.items():
    p.addConstraint(lambda x, val=part: x == val, (cell,))

p.getSolution()

KeyboardInterrupt: 

#### 2. Sudoku (Nikoli) – 10 points

> Place the digits 1 through 9 into the empty squares (one per square) so that each digit appears exactly once in
each of the following regions: the nine rows, the nine columns, and the nine outlined 3x3 regions.

> Answer: Enter the seventh row of digits, followed by the seventh column of digits.

In [4]:
# credit: https://simplapi.wordpress.com/2012/11/02/python-constraint-and-sudoku/

from constraint import Problem, InSetConstraint, AllDifferentConstraint
import math
 
def solveSudoku(size = 9, originalGame = None):
    """ Solving Sudoku of any size """
    sudoku = Problem()
 
    #Defining size of row/col
    rows = range(size)
    cols = range(size)
 
    #Creating board
    board = [(row, col) for row in rows for col in cols]
    #Defining game variable, a single range will be enough
    sudoku.addVariables(board, range(1, size + 1))
 
    #Row set
    rowSet = [list(zip([el] * len(cols), cols)) for el in rows]
    colSet = [list(zip(rows, [el] * len(rows))) for el in cols]
  
    #The original board is not empty, we add that constraint to the list of constraint
    if originalGame is not None:
        for i in range(0, size):
            for j in range(0, size):
                #Getting the value of the current game
                o = originalGame[i][j]
                #We apply constraint when the number is set only
                if o > 0:
                    #We get the associated tuple
                    t = (rows[i],cols[j])
                    #We set a basic equal constraint rule to force the system to keep that variable at that place
                    sudoku.addConstraint(lambda var, val=o: var == val, (t,))
 
    #The constraint are like that : and each row, and each columns, got same final compute value
 
    for row in rowSet:
        sudoku.addConstraint(AllDifferentConstraint(), row)
    for col in colSet:
        sudoku.addConstraint(AllDifferentConstraint(), col)
 
    #Every sqrt(size) (3x3 box constraint) got same sum
    sqSize = int(math.floor(math.sqrt(size)))
 
    #xrange allow to define a step, here sq (wich is sq = 3 in 9x9 sudoku)
    for i in range(0,size,sqSize):
        for j in range(0,size,sqSize):
            #Computing the list of tuple linked to that box
            box = []
            for k in range(0, sqSize):
                for l in range(0, sqSize):
                    #The tuple i+k, j+l is inside that box
                    box.append( (i+k, j+l) )
            #Compute is done, now we can add the constraint for that box
            sudoku.addConstraint(AllDifferentConstraint(), box)
 
    #Computing and returning final result
    return sudoku.getSolution()
 
 

rg = 9
initValue = [[0, 1, 0, 5, 0, 0, 0, 8, 0],
             [0, 2, 3, 4, 0, 0, 7, 0, 0],
             [0, 0, 0, 0, 0, 6, 0, 0, 0],
             [0, 0, 8, 1, 0, 0, 0, 0, 5],
             [0, 7, 0, 0, 0, 0, 0, 4, 0],
             [6, 0, 0, 0, 0, 3, 2, 0, 0],
             [0, 0, 0, 6, 0, 0, 0, 0, 0],
             [0, 0, 7, 0, 0, 4, 3, 2, 0],
             [0, 8, 0, 0, 0, 5, 0, 1, 0]]

res = solveSudoku(rg, initValue)
if res is not None:
#     for i in range(0, rg):
#         for j in range(0, rg):
#             print(res[i, j], end='')
#         print()
#     print()
    
    print([res[6, i] for i in range(rg)])
    print([res[i, 6] for i in range(rg)])
else:
    print("No result to show")

[3, 4, 1, 6, 2, 8, 5, 9, 7]
[4, 7, 1, 9, 8, 2, 5, 3, 6]


#### 3. Missing Operation KenKen® (Nextoy LLC) – 10 points

> Standard KenKen rules apply, but with the
operations missing.
Fill each square with a digit between 1 and 6,
not repeating a digit in any row or column. Each
outlined region (cage) includes a target number,
which is the result of some one arithmetic
operation applied to all the digits inside the
cage.
Cages with one square have no operation: the
missing digit is the same as the target.
Subtraction and division only appear in cages
with two squares, and the digits may appear in
any order. For cages with more that two
squares, the digits are added or multiplied
individually.

In [5]:
from constraint import Problem, AllDifferentConstraint
from functools import partial, reduce
from operator import mul

size = 6

kenken = Problem()

#Defining size of row/col
rows = range(size)
cols = range(size)

#Creating board
board = [(row, col) for row in rows for col in cols]
#Defining game variable, a single range will be enough
kenken.addVariables(board, range(1, size + 1))

#Row set
rowSet = [list(zip([el] * len(cols), cols)) for el in rows]
colSet = [list(zip(rows, [el] * len(rows))) for el in cols]

for row in rowSet:
    kenken.addConstraint(AllDifferentConstraint(), row)
for col in colSet:
    kenken.addConstraint(AllDifferentConstraint(), col)

regions = [  (4,  [(0, 0), (0, 1)]),
             (40, [(0, 2), (0, 3), (1, 3)]),
             (3,  [(0, 4), (0, 5)]),
             (12, [(1, 0), (1, 1), (1, 2)]),
             (3,  [(1, 4), (1, 5), (2, 5)]),
             (15, [(2, 0), (3, 0), (3, 1)]),
             (2,  [(2, 1)]),
             (3,  [(2, 2), (3, 2)]),
             (10, [(2, 3), (2, 4)]),
             (21, [(3, 3), (3, 4), (3, 5), (4, 4), (4, 5)]),
             (6,  [(4, 0), (4, 1), (4, 2)]),
             (20, [(4, 3), (5, 3), (5, 2)]),
             (2,  [(5, 0), (5, 1)]),
             (2,  [(5, 4), (5, 5)])]
    

def func(*cells, value=None):
    if len(cells) == 1:
        return cells[0] == value
    elif len(cells) == 2:
        return any([sum(cells) == value,
                    cells[0] - cells[1] == value,
                    cells[1] - cells[0] == value,
                    mul(*cells) == value,
                    cells[0] / cells[1] == value,
                    cells[1] / cells[0] == value])
    else:
        return any([sum(cells) == value,
                    reduce(mul, cells) == value])

for val, region in regions:
    kenken.addConstraint(partial(func, value=val), region)
    

res = kenken.getSolution()
if res is not None:
#     for i in range(0, size):
#         for j in range(0, size):
#             print(res[i, j], end='')
#         print()
#     print()
    print([res[4, i] for i in range(size)])
    print([res[i, 4] for i in range(size)])
else:
    print("No solution")

[3, 1, 2, 4, 6, 5]
[3, 1, 4, 5, 6, 2]


#### 4. Sum Thing (Ed Pegg Jr.) - 10 points

> Place each of the digits 0 through 9 into the empty circles (one per circle) so that the sum of the digits along each
line is equal to 13.

In [6]:
from constraint import Problem, AllDifferentConstraint, ExactSumConstraint
from functools import partial, reduce
from operator import mul

size = 10
line_sum = 13
board = range(size)
lines = [[0, 1, 7], [0, 5, 8], [0, 2, 4], [1, 2, 3], [1, 5, 9],
         [2, 6, 7], [3, 5, 7], [3, 4, 9], [4, 5, 6], [6, 8, 9]]

thing = Problem()
thing.addVariables(board, range(size))
thing.addConstraint(AllDifferentConstraint(), board)
for line in lines:
    thing.addConstraint(ExactSumConstraint(line_sum), line)

res = thing.getSolution()
if res is not None:
    print(', '.join([str(res[i]) for i in board]))
else:
    print("No solution")

0, 6, 5, 2, 8, 4, 1, 7, 9, 3


#### 7. Writer's Block (Craig Kasper) - 10 points

> Enter names into the crisscross grid across and down, one letter per square. Four names will not be used.
Use only the portions of the name that are fully capitalized, ignoring spaces and punctuation; names are sorted
and grouped by length.

> Answer: Enter the four missing capitalized names (ignore spaces and punctuation if any).

<img src='/files/2009_7_writers_block.png' />

In [7]:
from constraint import Problem, AllDifferentConstraint
from functools import partial

names = ['ALBEE', 'ALBOM', 'ALGER', 'BLAKE', 'FROST', 'GRIMM', 'MAMET',
         'PLATH', 'TYLER', 'WELTY', 'ASIMOV', 'BRONTE', 'CAPOTE', 'CHABON',
         'HORACE', 'JEWETT', 'KILMER', 'MAILER', 'MILLER', 'MILTON', 'ONEILL',
         'FAULKNER', 'MACLEISH', 'MCCARTHY', 'MELVILLE', 'MICHENER', 'HARPERLEE',
         'HEMINGWAY', 'JAMESAGEE', 'JKROWLING', 'NEILSIMON']

places_count = 27
places = range(places_count)

crisscross = Problem()
crisscross.addVariables(places, names)

crisscross.addConstraint(AllDifferentConstraint(), places)

place_sizes = {0:5, 1:5, 2:9, 3:6, 4:6, 5:8, 6:5, 7:8, 8:6, 9:6,
               10:8, 11:9, 12:9, 13:5, 14:6, 15:6, 16:5, 17:6, 18:6,
               19:6, 20:5, 21:8, 22:5, 23:5, 24:5, 25:6, 26:9}

for place, size in place_sizes.items():
    crisscross.addConstraint(lambda x, s=size: len(x) == s, (place, ))

# place1, index1, place2, index2
intersections = [(0, 0, 17, 0),
                 (4, 0, 17, 2),
                 (12, 3, 17, 5),
                 (23, 1, 4, 5),
                 (23, 4, 12, 8),
                 (21, 1, 12, 5),
                 (12, 0, 15, 0),
                 (16, 4, 25, 0),
                 (19, 2, 24, 1),
                 (21, 6, 24, 3),
                 (19, 4, 26, 7),
                 (26, 1, 11, 7),
                 (11, 0, 10, 6),
                 (11, 3, 18, 2),
                 (18, 5, 12, 3),
                 (12, 5, 21, 1),
                 (22, 1, 11, 5),
                 (11, 7, 26, 1),
                 (9, 1, 2, 3),
                 (9, 3, 8, 0),
                 (9, 5, 10, 4),
                 (13, 0, 2, 6),
                 (8, 3, 13, 2),
                 (10, 7, 13, 4),
                 (7, 0, 6, 4),
                 (1, 2, 6, 0),
                 (10, 0, 5, 0),
                 (10, 2, 7, 2),
                 (14, 0, 3, 0),
                 (14, 2, 5, 2),
                 (14, 4, 7, 4),
                 (20, 0, 3, 3),
                 (20, 2, 5, 5),
                 (20, 4, 7, 7),
                ]

def intersection_func(p1, p2, i1=None, i2=None):
    try:
        return p1[i1] == p2[i2]
    except IndexError:
        return False

for place1, idx1, place2, idx2 in intersections:
    crisscross.addConstraint(partial(intersection_func, i1=idx1, i2=idx2), (place1, place2))

res = crisscross.getSolution()

# print(res or "No solution")

set(names) - set(res.values())

{'ALBOM', 'ASIMOV', 'JKROWLING', 'MACLEISH'}

#### 9. Coordinate Pairs (Erich Friedman) - 10 points

> Connect pairs of points (using each point only
once), so that the corresponding segments can
be separated into two sets, each set containing
segments with the same length.

<img src='/files/2009_9_coordinate_pairs.png' />

In [8]:
from itertools import combinations, chain
from math import sqrt, pow
from collections import defaultdict
from decimal import Decimal


coords = {'A':(1,6), 'B':(2,6), 'C':(2,5), 'D':(5,5), 'E':(0,4), 'F':(2,4), 'G':(3,4),
          'H':(4,4), 'I':(6,4), 'J':(5,3), 'K':(0,2), 'L':(0,1), 'M':(1,1), 'N':(4,0)}

len_coords = len(coords)
len_segment_vars = int(len(coords.keys())/2)

def get_length(p1, p2):
    return Decimal(sqrt(pow((p2[0] - p1[0]), 2) + \
                        pow((p2[1] - p1[1]), 2)))

segments_length_map = {f'{p1}{p2}':get_length(coords[p1], coords[p2]) \
                       for p1, p2 in list(combinations(coords.keys(), 2))}

segments_length_reverse_map = defaultdict(list)

for k, v in segments_length_map.items():
    segments_length_reverse_map[v].append(k)

def get_letter_sequence(length_pair):
    return list(chain(*map(segments_length_reverse_map.get, length_pair)))

def has_all_unique_letters(segs):
    seen = set(''.join(segs))
    return len(seen) == len_coords

for length_pair in combinations(segments_length_reverse_map.keys(), 2):
    segs = get_letter_sequence(length_pair)
    if has_all_unique_letters(segs):
        for comb in combinations(segs, len_segment_vars):
            if has_all_unique_letters(comb):
                print(sorted(comb))
                break
        

['AE', 'BI', 'CL', 'DG', 'FN', 'HK', 'JM']
